In [1]:
import pandas as pd
from skimage.segmentation import slic,quickshift
from skimage.segmentation import mark_boundaries
from skimage.util import img_as_float
from skimage import io
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
from skimage.future import graph
import networkx
import glob
import cv2

In [2]:
def get_filelist(img_loc):
    filelist = glob.glob(img_loc+'*.jpg')
    filelist.sort()
    return filelist

In [3]:
filelist=get_filelist("/home/sirzechlucifer/Work/CVPR_graphs/ISIC2018_input/")

In [4]:
labels = pd.read_csv('/home/sirzechlucifer/Work/CVPR_graphs/ISIC2018_target/ISIC2018_Task3_Training_GroundTruth.csv')

In [5]:
len(filelist)

10015

In [6]:
labels.head()

,image,MEL,NV,BCC,AKIEC,BKL,DF,VASC
0,ISIC_0024306,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,ISIC_0024307,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,ISIC_0024308,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,ISIC_0024309,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,ISIC_0024310,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
labels.groupby('MEL').count()

,image,NV,BCC,AKIEC,BKL,DF,VASC
MEL,,,,,,,
0.0,8902,8902,8902,8902,8902,8902,8902
1.0,1113,1113,1113,1113,1113,1113,1113


In [8]:
##BCC,DF,VASC,AKIEC
BCC=list()
DF=list()
VASC=list()
AKIEC=list()
for index,row in labels.iterrows():
    if row["BCC"]==1:
        BCC.append(row["image"])
    if row["DF"]==1:
        DF.append(row["image"])
    if row["VASC"]==1:
        VASC.append(row["image"])
    if row["AKIEC"]==1:
        AKIEC.append(row["image"])

In [9]:
len(BCC)

514

In [10]:
BCC[0]

'ISIC_0024331'

In [16]:
for i in BCC:
    originalImage = cv2.imread('/home/sirzechlucifer/Work/CVPR_graphs/ISIC2018_input/'+i+".jpg")
    flipVertical = cv2.flip(originalImage, 0)
    flipHorizontal = cv2.flip(originalImage, 1)
    flipBoth = cv2.flip(originalImage, -1)
    #cv2.imwrite("/home/sirzechlucifer/Work/CVPR_graphs/Augmented_image/"+i+"_ver.jpg", flipVertical) 
    #cv2.imwrite("/home/sirzechlucifer/Work/CVPR_graphs/Augmented_image/"+i+"_hor.jpg", flipHorizontal)
    cv2.imwrite("/home/sirzechlucifer/Work/CVPR_graphs/Augmented_image/"+i+"_both.jpg", flipBoth)

In [17]:
for i in DF:
    originalImage = cv2.imread('/home/sirzechlucifer/Work/CVPR_graphs/ISIC2018_input/'+i+".jpg")
    flipVertical = cv2.flip(originalImage, 0)
    flipHorizontal = cv2.flip(originalImage, 1)
    flipBoth = cv2.flip(originalImage, -1)
    cv2.imwrite("/home/sirzechlucifer/Work/CVPR_graphs/Augmented_image/"+i+"_ver.jpg", flipVertical) 
    cv2.imwrite("/home/sirzechlucifer/Work/CVPR_graphs/Augmented_image/"+i+"_hor.jpg", flipHorizontal)
    cv2.imwrite("/home/sirzechlucifer/Work/CVPR_graphs/Augmented_image/"+i+"_both.jpg", flipBoth)

In [18]:
len(DF)

115

In [19]:
for i in VASC:
    originalImage = cv2.imread('/home/sirzechlucifer/Work/CVPR_graphs/ISIC2018_input/'+i+".jpg")
    flipVertical = cv2.flip(originalImage, 0)
    flipHorizontal = cv2.flip(originalImage, 1)
    flipBoth = cv2.flip(originalImage, -1)
    cv2.imwrite("/home/sirzechlucifer/Work/CVPR_graphs/Augmented_image/"+i+"_ver.jpg", flipVertical) 
    cv2.imwrite("/home/sirzechlucifer/Work/CVPR_graphs/Augmented_image/"+i+"_hor.jpg", flipHorizontal)
    cv2.imwrite("/home/sirzechlucifer/Work/CVPR_graphs/Augmented_image/"+i+"_both.jpg", flipBoth)

In [20]:
for i in AKIEC:
    originalImage = cv2.imread('/home/sirzechlucifer/Work/CVPR_graphs/ISIC2018_input/'+i+".jpg")
    flipVertical = cv2.flip(originalImage, 0)
    flipHorizontal = cv2.flip(originalImage, 1)
    flipBoth = cv2.flip(originalImage, -1)
    cv2.imwrite("/home/sirzechlucifer/Work/CVPR_graphs/Augmented_image/"+i+"_ver.jpg", flipVertical) 
    cv2.imwrite("/home/sirzechlucifer/Work/CVPR_graphs/Augmented_image/"+i+"_hor.jpg", flipHorizontal)
    cv2.imwrite("/home/sirzechlucifer/Work/CVPR_graphs/Augmented_image/"+i+"_both.jpg", flipBoth)

In [21]:
aug_filelist=get_filelist("/home/sirzechlucifer/Work/CVPR_graphs/Augmented_image/")

In [22]:
len(aug_filelist)

2266

In [42]:
aug_da=list()
for i in aug_filelist:
    aug_da.append(aug_filelist[0].split("/")[6].split(".")[0])

In [36]:
aug_labels=list()
for i in aug_filelist:
    for j in AKIEC:
        if i.split("/")[6].split("_")[1] in j:
            aug_labels.append(3)
    for j in BCC:
        if i.split("/")[6].split("_")[1] in j:
            aug_labels.append(2)
    for j in VASC:
        if i.split("/")[6].split("_")[1] in j:
            aug_labels.append(6)
    for j in DF:
        if i.split("/")[6].split("_")[1] in j:
            aug_labels.append(5)

In [37]:
len(aug_labels)

2266

In [39]:
aug_list=list()
for i in aug_labels:
    if i==3:
        aug_list.append([0.0,0.0,0.0,1.0,0.0,0.0,0.0])
    if i==2:
        aug_list.append([0.0,0.0,1.0,0.0,0.0,0.0,0.0])
    if i==6:
        aug_list.append([0.0,0.0,0.0,0.0,0.0,0.0,1.0])
    if i==5:
        aug_list.append([0.0,0.0,0.0,0.0,0.0,1.0,0.0])

In [45]:
data={"image":aug_da,"label":aug_labels}
aug_dataset=pd.DataFrame(data)

In [46]:
aug_dataset.head()

,image,label
0,ISIC_0024318_both,5
1,ISIC_0024318_both,5
2,ISIC_0024318_both,5
3,ISIC_0024318_both,3
4,ISIC_0024318_both,3


In [52]:
MEL_new=list()
NV_new=list()
BCC_new=list()
AKIEC_new=list()
BKL_new=list()
DF_new=list()
VASC_new=list()
for i,row in aug_dataset.iterrows():
    if row["label"]==3:
        MEL_new.append(0.0)
        NV_new.append(0.0)
        BCC_new.append(0.0)
        AKIEC_new.append(1.0)
        BKL_new.append(0.0)
        DF_new.append(0.0)
        VASC_new.append(0.0)
    if row["label"]==2:
        MEL_new.append(0.0)
        NV_new.append(0.0)
        BCC_new.append(1.0)
        AKIEC_new.append(0.0)
        BKL_new.append(0.0)
        DF_new.append(0.0)
        VASC_new.append(0.0)
    if row["label"]==6:
        MEL_new.append(0.0)
        NV_new.append(0.0)
        BCC_new.append(0.0)
        AKIEC_new.append(0.0)
        BKL_new.append(0.0)
        DF_new.append(0.0)
        VASC_new.append(1.0)
    if row["label"]==5:
        MEL_new.append(0.0)
        NV_new.append(0.0)
        BCC_new.append(0.0)
        AKIEC_new.append(0.0)
        BKL_new.append(0.0)
        DF_new.append(1.0)
        VASC_new.append(0.0)

In [53]:
data={"image":aug_da,"MEL":MEL_new,"NV":NV_new,"BCC":BCC_new,"AKIEC":AKIEC_new,"BKL":BKL_new,"DF":DF_new,"VASC":VASC_new}
aug_dat=pd.DataFrame(data)

In [54]:
len(aug_da)

2266

In [55]:
len(MEL_new)

2266

In [56]:
aug_dat.head()

,image,MEL,NV,BCC,AKIEC,BKL,DF,VASC
0,ISIC_0024318_both,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,ISIC_0024318_both,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,ISIC_0024318_both,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,ISIC_0024318_both,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,ISIC_0024318_both,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [57]:
aug_dat.to_csv("augmented_dataset.csv",index=False)